In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install openai==0.28

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 6.2 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 1.76.0
    Uninstalling openai-1.76.0:
      Successfully uninstalled openai-1.76.0


In [ ]:
import pandas as pd
import openai

# Set your OpenAI API key
openai.api_key = "----"

In [ ]:
# Load your original posts
df = pd.read_csv('/content/drive/MyDrive/Project Tesla/tesla_solar_reddit_posts_2025.csv')

# Fill missing title/text safely
df['title'] = df['title'].fillna('')
df['text'] = df['text'].fillna('')

# Combine title and text into one field for analysis
df['content'] = df['title'] + ' ' + df['text']

# Remove very short posts
df = df[df['content'].str.len() > 10]

# Reset index
df = df.reset_index(drop=True)

print(f"Total posts after cleaning: {len(df)}")

Total posts after cleaning: 557


In [ ]:
# Summarization function
def summarize_post(post_text):
    prompt = f"Summarize the following Reddit post into 1-2 sentences, highlighting the main issue or event:\n\n{post_text}"
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are a helpful summarizer."},
            {"role": "user", "content": prompt}
        ],
        temperature=0.3,
        max_tokens=60
    )
    return response['choices'][0]['message']['content'].strip()

# Create summary column only if it doesn't exist yet
if 'summary' not in df.columns:
    df['summary'] = df['content'].apply(summarize_post)

In [ ]:
def create_prompt(existing_labels, post_text, similarity_threshold=70):
    if not existing_labels:
        return f"""
You are a smart assistant for categorizing Reddit posts related to Tesla Solar products.

Task:
- Suggest a new topic label for the following post.
- The label must be specific, informative, and actionable.
- It should clearly summarize the main issue or event in 3 to 7 words.
- Avoid generic labels like "Solar Discussion" or "Update."
- Focus on the real problem, experience, or event being discussed.

Post:
"{post_text}"

Respond ONLY in the format:
New Label: <your label>
"""

    label_list = '\n'.join([f"- {label}: {desc}" for label, desc in existing_labels.items()])

    return f"""
You are a smart assistant for categorizing Reddit posts related to Tesla Solar products.

Existing Labels:
{label_list}

New Post:
"{post_text}"

Task:
- If any existing label matches the post meaning with ≥ {similarity_threshold}% similarity, pick that label.
- If no existing label matches well enough, suggest a new label.
- The label must be specific, informative, and actionable.
- It should clearly summarize the main issue or event in 3 to 7 words.
- Avoid vague labels like "Feedback" or "Update."
- Focus on the real problem, experience, or event being discussed.

Respond ONLY in the format:
Existing Label: <label name>
or
New Label: <your new label suggestion>
"""

In [ ]:
# Send prompt to GPT and get clean answer
def ask_llm(prompt):
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": prompt}
        ],
        temperature=0.3,
        max_tokens=50  # short focused output
    )
    return response['choices'][0]['message']['content'].strip()

In [ ]:
# Initialize
labels = {}  # Store existing labels
assigned_labels = []  # Store assigned label for each post

# Loop through posts
for idx, post_text in enumerate(df['content']):
    prompt = create_prompt(labels, post_text)
    response = ask_llm(prompt)

    if response.startswith("Existing Label:"):
        label_name = response.replace("Existing Label:", "").strip()
    elif response.startswith("New Label:"):
        label_name = response.replace("New Label:", "").strip()
        labels[label_name] = post_text  # Save first example post as description
    else:
        label_name = f"Label_{idx}"
        labels[label_name] = post_text

    assigned_labels.append(label_name)

# Add labels to DataFrame
df['category'] = assigned_labels

print("Posts have been labeled!")


Posts have been labeled!


In [ ]:
# Save labeled posts to new CSV
df.to_csv('/content/drive/MyDrive/Project Tesla/tesla_solar_reddit_posts_labeled_2025.csv', index=False)

print("Final CSV saved with 'category' column!")


Final CSV saved with 'category' column!


In [ ]:
label_grouping = {
    # Installation Experience and Issues
    "Tesla Solar System Installation Experience": "Installation Experience and Issues",
    "Tesla Solar Panel Installation Challenges": "Installation Experience and Issues",
    "Tesla Solar Panel Installation Inquiry": "Installation Experience and Issues",
    "Tesla Solar Panel Installation Satisfaction": "Installation Experience and Issues",
    "Solar Roof Installation Inquiry": "Installation Experience and Issues",
    "Solar System Installation Documentation Requirement": "Installation Experience and Issues",
    "Tesla Solar Panel Installation Documentation Requirement": "Installation Experience and Issues",
    "Solar Panel Installation Decision Inquiry": "Installation Experience and Issues",
    "Solar Panel Installation Constraint: CalPac Roof Issue": "Installation Experience and Issues",

    # System Performance Problems
    "Tesla Solar System Performance Issues": "System Performance Problems",
    "Tesla Solar System Performance Concerns": "System Performance Problems",
    "Solar Panel Performance Anomaly": "System Performance Problems",
    "Solar Panel Production Anomaly": "System Performance Problems",
    "Solar Panel Performance Concern": "System Performance Problems",
    "Tesla Solar Panel Performance Concern": "System Performance Problems",
    "Solar Panel Production Interruption": "System Performance Problems",
    "Tesla Solar Panel Production Interruption": "System Performance Problems",
    "Solar Panel Production Optimization": "System Performance Problems",
    "Tesla Solar Panel Production Optimization": "System Performance Problems",
    "Solar Panel Shade Issue": "System Performance Problems",
    "Tesla Solar Panel Snow Accumulation Issue": "System Performance Problems",
    "Solar Panel Data Display Issue": "System Performance Problems",
    "Tesla Solar Panel Data Display Issue": "System Performance Problems",

    # Powerwall Topics
    "Powerwall Delay Frustration": "Powerwall Topics",
    "Powerwall Charging Optimization Strategy": "Powerwall Topics",
    "Powerwall Charging Strategy Optimization": "Powerwall Topics",
    "Powerwall Dependency Concern": "Powerwall Topics",
    "Powerwall Pricing Discrepancy": "Powerwall Topics",
    "Powerwall Alternative Inquiry": "Powerwall Topics",
    "Powerwall Installation Configuration Inquiry": "Powerwall Topics",
    "Powerwall 3 Configuration Inquiry": "Powerwall Topics",
    "Powerwall 3 String Configuration Inquiry": "Powerwall Topics",
    "Powerwall Expansion Inquiry": "Powerwall Topics",
    "Powerwall Expansion Pack Availability": "Powerwall Topics",
    "Powerwall Installation Code Violation": "Powerwall Topics",
    "Powerwall Charging Issue": "Powerwall Topics",

    # Cost & Pricing Issues
    "Tesla Powerwall Pricing Discrepancy": "Cost & Pricing Issues",
    "Tesla Solar Panel Lease Battery Compatibility": "Cost & Pricing Issues",
    "Solar System Cost Analysis": "Cost & Pricing Issues",
    "Solar Panel Quote": "Cost & Pricing Issues",

    # Tesla Electric Plans
    "Tesla Electric Plan Review": "Tesla Electric Plans",
    "Tesla Electric Plan Comparison": "Tesla Electric Plans",

    # Solar Panel Damage or Maintenance
    "Tesla Panel Hail Damage Concerns": "Solar Panel Damage or Maintenance",
    "Solar Panel Damage Inquiry": "Solar Panel Damage or Maintenance",
    "Solar Panel Post-Storm Troubleshooting": "Solar Panel Damage or Maintenance",
    "Solar Panel Maintenance Inquiry": "Solar Panel Damage or Maintenance",
    "Tesla Solar Panel Pest Abatement": "Solar Panel Damage or Maintenance",
    "Tesla Solar Panel Troubleshooting": "Solar Panel Damage or Maintenance",

    # Customer Support and Satisfaction
    "Tesla Solar Customer Service Experience": "Customer Support and Satisfaction",
    "Customer Satisfaction Inquiry": "Customer Support and Satisfaction",
    "Duplicate Powerwall Installation Email": "Customer Support and Satisfaction",

    # Solar Energy Sharing/Innovation
    "Solar Energy Sharing Marketplace Concept": "Solar Energy Sharing/Innovation",
    "Solar Energy Sharing App Concept": "Solar Energy Sharing/Innovation",
    "Solar and Starlink Connectivity Success": "Solar Energy Sharing/Innovation",
    "Tesla Model Y Sister Acquisition": "Solar Energy Sharing/Innovation",

    # Design Consultation and Data Display
    "Solar System Design Consultation Inquiry": "Design Consultation and Data Display",
    "Solar Panel Design Discrepancy": "Design Consultation and Data Display",
    "Tesla Solar Panel Design Discrepancy": "Design Consultation and Data Display",
    "Solar Panel Data Reporting Inquiry": "Design Consultation and Data Display",

    # Tax and Insurance Concerns
    "Solar Panel Tax Credit Discrepancy": "Tax and Insurance Concerns",
    "Homeowners Insurance for Solar Roof": "Tax and Insurance Concerns"
}

df['dashboard_category'] = df['category'].map(label_grouping).fillna(df['category'])

In [ ]:
# Save labeled posts to new CSV
df.to_csv('/content/drive/MyDrive/Project Tesla/tesla_solar_reddit_posts_GroupLabeled_2025.csv', index=False)

print("Final CSV saved with 'category' column!")

Final CSV saved with 'category' column!
